### Mini Project - Yawn Detector and Counter.
**N.b: Can be used in drowsiness detection systems**

In [4]:
import cv2
import dlib
import numpy as np


PREDICTOR_PATH = r'C:\Users\princ\Documents\Mini openCV projects\shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(PREDICTOR_PATH)    # the facial landmark detector
detector = dlib.get_frontal_face_detector()         # the face detector

In [5]:
 def get_landmarks(im):
    """
    gets the facial landmarks from the image passed
    
    Arg:
        The input image
        
    Returns:
        A 2D-array or matrix holding the x and y cords of each landmark
    """
    rects = detector(im, 1)
    
    if len(rects) > 1:                      # if the number of faces is gretaer than one.
        return 'error'
    if len(rects) == 0:                     # if the number of faces detected is zero.
        return 'error'
    return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])


def annotate_landmarks(im, landmarks):
    """
    annotates the landmark passed using it's index
    """
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos, fontFace= cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, fontScale=0.4, color=(0,0, 255))
        cv2.circle(im,pos,3, color=(0, 255, 255))
    return im




def get_top_lip(landmarks):
    """
    gets the top lip postion from the facial landmarks and use the mean pos
    as the top lip position.
    
    Args:
        landmarks: the face facial landmarks
        
    Returns:
        The top lip position
    """
    top_lip_pts = []
    for i in range(50, 53):
        top_lip_pts.append(landmarks[i])
    for i in range(61, 64):
        top_lip_pts.append(landmarks[i])
    top_lip_all_pts = np.squeeze(np.asarray(top_lip_pts))
    top_lip_mean = np.mean(top_lip_pts, axis=0)
    return int(top_lip_mean[:, 1])



# function that gets the bottom lip using the bottom lip landmarks
def get_bottom_lip(landmarks):
    """
    gets the bottom lip postion from the facial landmarks and use the mean pos
    as the bottom lip position.
    
    Args:
        landmarks: the face facial landmarks
        
    Returns:
        The bottom lip position
    """
    bottom_lip_pts = []
    for i in range(65, 68):
        bottom_lip_pts.append(landmarks[i])
    for i in range(56, 59):
        bottom_lip_pts.append(landmarks[i])
    bottom_lip_all_pts = np.squeeze(np.asarray(bottom_lip_pts))
    bottom_lip_mean = np.mean(bottom_lip_pts, axis=0)
    return int(bottom_lip_mean[:, 1])


def mouth_open(image):
    """
    Calculates the distance between the subjects top lips and bottom lips pos
    Args:
        image: The input image or frame as case may be
        
    Returns:
        image with landmarks annotated and the lip distance
    """
    landmarks = get_landmarks(image)
    
    if landmarks == 'error':
        return image, 0
    
    image_with_landmarks = annotate_landmarks(image, landmarks)
    top_lip_center = get_top_lip(landmarks)
    bottom_lip_center = get_bottom_lip(landmarks)
    
    lip_distance = abs(top_lip_center - bottom_lip_center)
    return image_with_landmarks, lip_distance



In [9]:
cap = cv2.VideoCapture(0)
yawns = 0     # keeps track of number of times subject has yawned
yawn_status = False   # keeps track of whether subject is yawning or not


while True:
    ret, frame = cap.read()
    image_landmarks, lip_distance = mouth_open(frame)
    prev_yawn_status = yawn_status
    yawn_lip_distance = 45
    
    if lip_distance > yawn_lip_distance:
        yawn_status = True
        cv2.putText(frame, 'Subject is Yawning', (50, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
    else:
        yawn_status = False
            
    if prev_yawn_status == True and yawn_status == False:
        yawns += 1
            
    
    output_text = 'Yawn Count: ' + str(yawns)
    cv2.putText(frame, output_text, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 127),2)
    
    cv2.imshow('Live landmarks', image_landmarks)
    cv2.imshow('Yawn Detector', frame)
        
    if cv2.waitKey(1) == 13:
        break
     
    
cap.release()
cv2.destroyAllWindows()